In [5]:
import threading
import time
import random

def print_names():
  for name in ("John", "Kate", "Mike", "Alex", "Ann"):
    print(name)
    time.sleep(0.2)

def print_age(min_sleep, max_sleep):
  for _ in range(5):
    print(random.randint(1,20))
    time.sleep(random.uniform(min_sleep, max_sleep))
    
# print_names()
# print_age()

In [6]:
# Create Thread object
t1 = threading.Thread(target = print_names)
t2 = threading.Thread(target = print_age, args=(0.2, 1))

# To make them do something:
t1.start()
t2.start()

# Wait for them to complete.
# Join means Joining them back up with the Main thread
t1.join()
t2.join()

John16

Kate
Mike
10
Alex
Ann
11
3
13


In [ ]:
"""Python threads aren't truly parallel, but are still useful for I/0 bound tasks""" 
import threading
import requests
from pathlib import Path

def download_file(url, filename):
  print(f"Downloading {url} to {filename}")
  response = requests.get(url)
  Path(filename).write_bytes(response.content )
  print(f"Finished downloading {filename}")

base_ur1 = "https://raw.githubusercontent.com/JacobCallahan/Understanding/master/Python/file_io"
urls = [
      f"{base_url}/binary_file",
      f"{base_url}/files.py",
      f"{base_url}/names.txt",
      f"{base_url}/new_file.txt",
]

threads = []

for url in urls:
  filename = "downloads/" + url.split("/")[-1]
  t = threading.Thread(target = download_file, args = (url, filename))
  t.start()
  threads.append(t)
  
[t.join() for t in threads]

In [ ]:
"""Threadpools are an easy and convenient way to run in threads""" 
import concurrent.futures
import requests
import time

def get_status(url):
  print(f"Getting status of {url}")
  start_time = time.monotonic()
  response = requests.get(ur1)
  total_time = time.monotonic() - start_time
  print(f"Finished getting status of {url} in {total_time:.2f} seconds")
  return url, response.status_code

urls = [
"https://www.google.com",
"https://www.facebook.com",
"https://www.twitter.com",
"https://ww.github.com",
"https://ww.linkedin.com"
]

# Use context manager:
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
  # Future represents a placeholder for result
  futures = []
  # Creating threads
  for url in urls:
    future = executor.submit(get_status, url)
    futures.append(future)
    
  # Starting threads. Gives us an iterator that yields as futures complete.
  for future in concurrent.futures.as_completed(futures):
    try:
      url, code = future.result()
      print(f"status code for {url} is {code}")
    except Exception as e:
      print(f"Task failed: {e}")


In [ ]:
"""Threadpools are an easy and convenient way to run in threads""" 
import concurrent.futures
import requests
import time
from pathlib import Path
import threading

STATUS_REPORT = Path("status_report.txt")
STATUS_REPORT.write_text("")
REPORT_LOCK = threading.Lock()

def get_status(url):
  REPORT_LOCK.acquire()
  current_text = STATUS_REPORT.read_text()
  print(f"Getting status of {url}")
  start_time = time.monotonic()
  response = requests.get(ur1)
  total_time = time.monotonic() - start_time
  print(f"Finished getting status of {url} in {total_time:.2f} seconds")
  STATUS_REPORT.write_text(f"{current_text}{url}: {response.status_code}\n")
  REPORT_LOCK.release()
  return url, response.status_code

# Alternative with context manager for lock(), but makes it sequential due to lock.
def get_status(url):
  with REPORT_LOCK:
    current_text = STATUS_REPORT.read_text()
    print(f"Getting status of {url}")
    start_time = time.monotonic()
    response = requests.get(ur1)
    total_time = time.monotonic() - start_time
    print(f"Finished getting status of {url} in {total_time:.2f} seconds")
    STATUS_REPORT.write_text(f"{current_text}{url}: {response.status_code}\n")
  return url, response.status_code

# An improvement by collapsing the lock only for the text it needs to be in:
def get_status(url):
  print(f"Getting status of {url}")
  start_time = time.monotonic()
  response = requests.get(ur1)
  total_time = time.monotonic() - start_time
  print(f"Finished getting status of {url} in {total_time:.2f} seconds")
  with REPORT_LOCK:
    current_text = STATUS_REPORT.read_text()
    STATUS_REPORT.write_text(f"{current_text}{url}: {response.status_code}\n")
  return url, response.status_code


urls = [
"https://www.google.com",
"https://www.facebook.com",
"https://www.twitter.com",
"https://ww.github.com",
"https://ww.linkedin.com"
]

# Use context manager:
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
  # Future represents a placeholder for result
  futures = []
  # Creating threads
  for url in urls:
    future = executor.submit(get_status, url)
    futures.append(future)
    
  # Starting threads. Gives us an iterator that yields as futures complete.
  for future in concurrent.futures.as_completed(futures):
    try:
      url, code = future.result()
      print(f"status code for {url} is {code}")
    except Exception as e:
      print(f"Task failed: {e}")
            
            

In [ ]:
""" Deadlocking - where one thread is waiting for another thread to finish, but that thread is waiting for some other / original thread to finish, hence never finishing."""
import threading

LOCK_1 = threading. Lock()
LOCK_2 = threading. Lock()

def thread1():
  LOCK_1.acquire()
  print("Thread 1 acquired lock1")
  LOCK_2.acquire()
  print("Thread 1 acquired lock2")
  LOCK_2.release()
  LOCK_1.release()

def thread2():
  LOCK_2.acquire()
  print("Thread 2 acquired lock2")
  LOCK_1.acquire()
  print ("Thread 2 acquired lock1")
  LOCK_1.release()
  LOCK_2.release()

# Will likely deadlock as they are trying to acquire same locks. 
t1 = threading.Thread(target = thread1)
t2 = threading.Thread(target = thread2)